# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys_Elyssa import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("my_data_Elyssa.csv")
df.head()

,City,Country,Latitude,Longitude,Date,Wind_Speed,Pressure,Humidity,Temperature,Cloudiness
0,Arrondissement de Figeac,FR,44.7500,1.9167,1.644208e+09,10.29,1025.0,97.0,42.19,100.0
1,Airai,TL,-8.9266,125.4092,1.644208e+09,2.59,1010.0,67.0,74.08,47.0
2,Mataura,NZ,-46.1927,168.8643,1.644208e+09,4.63,1018.0,38.0,60.48,6.0
3,Bereda,ES,43.2686,-7.5406,1.644208e+09,1.74,1034.0,98.0,38.66,60.0
4,Yambio,SS,4.5721,28.3955,1.644208e+09,4.25,1012.0,25.0,63.66,94.0


In [3]:
df_sub = df.dropna().reset_index(drop=True)
df_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City         54 non-null     object 
 1   Country      54 non-null     object 
 2   Latitude     54 non-null     float64
 3   Longitude    54 non-null     float64
 4   Date         54 non-null     float64
 5   Wind_Speed   54 non-null     float64
 6   Pressure     54 non-null     float64
 7   Humidity     54 non-null     float64
 8   Temperature  54 non-null     float64
 9   Cloudiness   54 non-null     float64
dtypes: float64(8), object(2)
memory usage: 4.3+ KB


In [4]:
df_sub.describe()

,Latitude,Longitude,Date,Wind_Speed,Pressure,Humidity,Temperature,Cloudiness
count,54.000000,54.000000,5.400000e+01,54.000000,54.000000,54.000000,54.000000,54.000000
mean,18.318335,0.931906,1.644208e+09,9.735926,1013.851852,78.981481,43.196481,60.240741
std,41.355865,92.972399,1.147091e+02,11.389862,14.762788,17.862674,34.812381,38.404469
min,-54.800000,-175.200000,1.644207e+09,0.000000,967.000000,25.000000,-40.050000,0.000000
25%,-21.318875,-71.116275,1.644208e+09,4.007500,1011.000000,71.250000,18.115000,27.250000
50%,22.588200,-2.811950,1.644208e+09,7.055000,1014.000000,78.500000,60.470000,72.000000
75%,54.743500,83.644500,1.644208e+09,11.500000,1023.000000,94.750000,70.825000,100.000000
max,78.218600,168.864300,1.644208e+09,61.000000,1034.000000,100.000000,88.000000,100.000000


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [6]:
#heatmap
locations = df_sub.loc[:, ["Latitude", "Longitude"]]

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    locations, weights=df_sub['Humidity'],
    max_intensity=20, point_radius=5
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# A max temperature lower than 80 degrees but higher than 70.


# Wind speed less than 10 mph.


# Zero cloudiness

mask = (df_sub.Temperature < 80) & (df_sub.Temperature > 70) & (df_sub.Wind_Speed < 10) & (df.Cloudiness == 0)

good_cities = df_sub.loc[mask]
good_cities

,City,Country,Latitude,Longitude,Date,Wind_Speed,Pressure,Humidity,Temperature,Cloudiness
33,Hobart,AU,-42.8794,147.3294,1.644208e+09,1.01,1023.0,46.0,76.73,20.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# geocoordinates
target_coordinates = f"{20.8947}, {-156.4700}"
target_search = ""

target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

places_data = response.json()

print(places_data["results"][0]["name"])
print(places_data["results"][0]["vicinity"])
print(places_data["results"][0]["rating"])

Maui Seaside Hotel
100 West Kaahumanu Avenue, Kahului
3.8


In [9]:
hotel_names = []
ratings = []
addresses = []
cities = []
countries = []
lats = []
longs = []

for _, row in good_cities.iterrows():
    print(row.City)

    # geocoordinates
    target_coordinates = f"{row.Latitude}, {row.Longitude}"
    target_search = ""

    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    places_data = response.json()

    hotel = places_data["results"][0]["name"]
    address = places_data["results"][0]["vicinity"]
    rating = places_data["results"][0]["rating"]
    
    # put it all in a list
    hotel_names.append(hotel)
    ratings.append(rating)
    addresses.append(address)
    cities.append(row.City)
    countries.append(row.Country)
    lats.append(row.Latitude)
    longs.append(row.Longitude)

Hobart


In [10]:
hotel_df = pd.DataFrame()
hotel_df['City'] = cities
hotel_df['Country'] = countries
hotel_df['Latitude'] = lats
hotel_df['Longitude'] = longs
hotel_df['Hotel'] = hotel_names
hotel_df['Address'] = addresses
hotel_df['Rating'] = ratings

hotel_df

,City,Country,Latitude,Longitude,Hotel,Address,Rating
0,Hobart,AU,-42.8794,147.3294,Mantra on Collins Hobart,"58 Collins Street, Hobart",4.2


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>Address</dt><dd>{Address}</dd>
<dt>Rating</dt><dd>{Rating}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Latitude", "Longitude"]]

In [14]:
# create layers
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
heatmap_layer = gmaps.heatmap_layer(
    locations, weights=df_sub['Humidity'],
    max_intensity=20, point_radius=5
)


fig = gmaps.figure()

In [15]:
# Add marker layer ontop of heat map
fig.add_layer(marker_layer)
fig.add_layer(heatmap_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))